In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import sys
import random
sys.path.append('../input/llm-20-questions/llm_20_questions/')
import keywords
from kaggle_environments import make

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
my_json = json.loads(keywords.KEYWORDS_JSON)
json_country = my_json[0]
json_city = my_json[1]
json_landmark = my_json[2]
import pprint

pprint.pprint(json_country['words'][:5])
pprint.pprint(json_city['words'][:5])
pprint.pprint(json_landmark['words'][:5])


In [ ]:
df_country = pd.json_normalize(json_country['words'])
df_city = pd.json_normalize(json_city['words'])
df_landmark = pd.json_normalize(json_landmark['words'])
print(df_country.head())
print(df_city.head())
print(df_landmark.head())

In [ ]:
# Combine all keywords into a single list
all_keywords = (
    list(df_country) +
    list(df_city) +
    list(df_landmark)
)
df = pd.DataFrame(all_keywords)
# Select 20 random keywords for questions
random_keywords = df.sample()
random_keywords

In [ ]:
# Define the agent function using the dataframes
class AgentWithQuestions:
    def __init__(self, df_country, df_city, df_landmark, questions):
        self.df_country = df_country
        self.df_city = df_city
        self.df_landmark = df_landmark
        self.questions = questions
        self.question_index = 0

    def __call__(self, obs, cfg):
        if obs.turnType == "ask":
            if self.question_index < len(self.questions):
                response = self.questions[self.question_index]
                print(f"Agent asks: {response}")
                self.question_index += 1
            else:
                response = "Is it a duck?"  # Fallback question if we run out
                print(f"Agent asks: {response}")
        elif obs.turnType == "guess":
            response = "country"  # Implement more complex guessing logic here
            print(f"Agent guesses: {response}")
        elif obs.turnType == "answer":
            entity = obs.entity.lower()
            if entity in self.df_country['word'].values:
                response = "country"
            elif entity in self.df_city['word'].values:
                response = "city"
            elif entity in self.df_landmark['word'].values:
                response = "landmark"
            else:
                response = "no"
            print(f"Agent answers: {response} to entity {entity}")
        return response

In [ ]:
# Initialize the agent
agent_instance = AgentWithQuestions(df_country, df_city, df_landmark, random_keywords)

# Define agent function wrapper
def agent_fn(obs, cfg):
    return agent_instance(obs, cfg)

# Configuration for the environment
debug_config = {
    'episodeSteps': 61,    # initial step plus 3 steps per round (ask/answer/guess)
    'actTimeout': 60,      # agent time per round in seconds; default is 60
    'runTimeout': 1200,    # max time for the episode in seconds; default is 1200
    'agentTimeout': 3600   # obsolete field; default is 3600
}

# Initialize the environment
env = make(environment="llm_20_questions", configuration=debug_config, debug=True)

# Run the game
game_output = env.run(agents=[agent_fn, agent_fn, agent_fn, agent_fn])

# Render the game output
env.render(mode="ipython", width=1080, height=700)